In [1]:
data_var = '2023-12-16'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
from datetime import date, timedelta
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
9927,2023-12-16,Brasil Nbb,16:00,Brasília,Paulistano,3.61,1.26,150.5,1.86,1.84,7.5,2.00,1.72,MZ63jW9i,0.277008,0.793651,0.537634,0.543478,0.070659,0.4,0.4,NaN,NaN,335.380,259.908942,0.774969,114.746,37.440728,0.326292,169.368,140.908,205.92,109.35,0.0,0.0,0.0,0.0,0.682423,0.007644,0.106446,-2.81,-0.562,-4.644128,0.0,0.0,0.0,1.48,0.296,0.878378,0.0,0.0,0.0
9928,2023-12-16,Brasil Nbb,17:00,Corinthians Paulista,Pato,1.32,3.20,155.5,1.81,1.85,-8.5,2.04,1.69,0Q57kjPc,0.757576,0.312500,0.552486,0.540541,0.070076,0.8,0.6,NaN,NaN,165.790,87.336074,0.526787,278.238,155.736265,0.559723,152.856,213.180,307.65,244.12,0.0,0.0,0.0,0.0,0.588213,0.015456,0.132701,-0.62,-0.124,-2.580645,0.0,0.0,0.0,-2.66,-0.532,-4.135338,0.0,0.0,0.0
9929,2023-12-16,Brasil Nbb,17:00,Pinheiros,Minas,3.12,1.33,157.5,1.80,1.86,5.5,2.08,1.67,rkeClAv4,0.320513,0.751880,0.555556,0.537634,0.072393,0.0,0.4,NaN,NaN,168.236,56.455315,0.335572,111.162,25.518162,0.229558,142.564,117.462,160.60,131.52,0.0,0.0,0.0,0.0,0.568863,0.023184,0.154621,0.40,0.080,26.500000,0.0,0.0,0.0,0.58,0.116,2.844828,0.0,0.0,0.0
9930,2023-12-16,Espanha Acb,14:00,Manresa,Tenerife,1.83,1.98,167.5,1.85,1.95,-2.5,2.07,1.70,fDtH7eYI,0.546448,0.505051,0.540541,0.512821,0.051499,0.0,0.0,NaN,NaN,326.530,202.968455,0.621592,229.016,123.884364,0.540942,0.000,0.000,143.42,338.00,0.0,0.0,0.0,0.0,0.055678,0.037216,0.138795,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
9931,2023-12-16,Espanha Acb,14:00,Unicaja,Granada,1.11,6.70,168.5,1.85,1.91,-14.5,2.07,1.70,rwwD8yJC,0.900901,0.149254,0.540541,0.523560,0.050155,0.0,0.0,NaN,NaN,177.674,153.407190,0.863419,310.126,179.720684,0.579509,0.000,0.000,102.83,600.30,0.0,1.0,0.0,0.0,1.012222,0.022567,0.138795,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10240,2023-12-16,Eua Ncaa,21:00,Wichita State,Southern Illinois,1.49,2.74,141.5,1.91,1.91,-6.5,2.10,1.67,ALp5Iiyo,0.671141,0.364964,0.523560,0.523560,0.036104,0.0,0.0,NaN,NaN,133.932,37.303157,0.278523,125.202,26.645687,0.212822,0.000,0.000,138.40,143.04,0.0,0.0,0.0,0.0,0.417912,0.000000,0.161303,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
10241,2023-12-16,Eua Ncaa,22:00,Alabama A&M,Chattanooga Mocs,4.44,1.23,149.5,1.91,1.91,7.5,2.00,1.71,8MXs6Fgn,0.225225,0.813008,0.523560,0.523560,0.038233,0.0,0.0,NaN,NaN,654.090,366.147594,0.559782,118.712,49.346646,0.415684,0.000,0.000,524.80,178.40,1.0,0.0,0.0,0.0,0.800639,0.000000,0.110545,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
10242,2023-12-16,Eua Ncaa,23:30,N. Carolina A&T,Texas Southern,4.80,1.20,144.5,1.91,1.91,8.5,2.10,1.67,fHp4Qg7T,0.208333,0.833333,0.523560,0.523560,0.041667,0.0,0.2,NaN,NaN,636.190,592.875698,0.931916,445.122,268.989678,0.604306,0.000,445.122,158.72,745.50,0.0,1.0,0.0,0.0,0.848528,0.000000,0.161303,0.00,0.000,inf,0.0,0.0,0.0,-5.00,-1.000,-0.200000,0.0,0.0,0.0
10243,2023-12-16,Europa Wbbl (Fem.),13:00,TTT Riga F,Klaipedos Neptunas F,1.06,7.80,140.5,1.82,1.85,-22.5,1.98,1.73,EXNW7dVa,0.943396,0.128205,0.549451,0.540541,0.071601,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,64.05,0.0,0.0,0.0,1.0,1.075824,0.011560,0.095297,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0


## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
7,00:00,Eua Nba,Phoenix Suns,New York Knicks,228.5,1.81,1.0000,Over
11,22:00,Eua Nba,Miami Heat,Chicago Bulls,215.5,1.83,1.0000,Over
13,22:30,Eua Nba,Golden State Warriors,Brooklyn Nets,232.5,1.80,1.0000,Over
14,23:00,Eua Nba,Denver Nuggets,Oklahoma City Thunder,235.5,1.84,1.0000,Over
107,16:00,Eua Ncaa,Mercer,Florida Gulf Coast,135.5,2.00,0.8363,Over
264,13:00,República Tcheca Zbl Feminina,USK Prague F,Trutnov F,149.5,1.87,0.5281,Over
